### kNN streaming data
- temporal
- spectral

In [ ]:
#DOMAIN = 'temporal' 
DOMAIN = 'spectral'

from river import preprocessing
from river import neighbors, utils, evaluate, stream

import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sb

import os
import sys
sys.path.append('../')
from vibrodiagnostics import selection, models, discovery
from vibrodiagnostics.selection import METADATA_COLUMNS_ALL
from vibrodiagnostics.models import (
    fault_labeling, filter_out_metadata_columns, project_classifier_map_plot
)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import random
import functools
from collections import defaultdict
from sklearn import metrics as skmetrics
from tqdm.notebook import tqdm
from river import metrics


FEATURES_PATH =  '../../datasets/features_data/'
FAULT_CLASSES = {
    'A': {
        'normal': 'normal',
        'imbalance': 'imbalance',
        'horizontal-misalignment': 'misalignment',
        'vertical-misalignment': 'misalignment',
        'underhang-outer_race': 'outer race fault',
        'underhang-cage_fault': 'cage fault',
        'underhang-ball_fault': 'ball fault'
    },
    'B': {
        'normal': 'normal',
        'imbalance': 'imbalance',
        'horizontal-misalignment': 'misalignment',
        'vertical-misalignment': 'misalignment',
        'overhang-cage_fault': 'cage fault',
        'overhang-ball_fault': 'ball fault',
        'overhang-outer_race': 'outer race fault'
    }
}

TD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_FEATURES_PATH)
FD_FEATURES = os.path.join(FEATURES_PATH, selection.FREQ_FEATURES_PATH)
TD_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_AND_FREQ_FEATURES_PATH)

In [ ]:

def knn_online_learn(X, Y, label='fault', window_len=1, learn_skip=0, clusters=False):
    # Buffer true samples for learning for later: simulate delayed annotation
    learning_window = []

    # Model consists of scaler to give approximately same weight to all features and kNN
    scaler = preprocessing.MinMaxScaler() 
    knn = neighbors.KNNClassifier(n_neighbors=5)

    scores = []                 # List of tuples with accuracy, precision and recall score on each iteration
    v_true = []                 # Append y true sample on each iteration
    v_predict = []              # Append y predicted sample on each iteration

    skipping = 0
    started = False
    order_saved = []
    X['fault'] = Y

    for idx, row in tqdm(X.iterrows()):
        x = {k: v for k, v in dict(row).items() if k not in METADATA_COLUMNS_ALL}

        x_scaled = scaler.learn_one(x).transform_one(x)
        y_true = row['fault']
        learning_window.append((x_scaled, y_true))

        if started:
            # Predict sample after at least one example has been learned
            y_predict = knn.predict_one(x_scaled)
            v_true.append(y_true)
            v_predict.append(y_predict)
            order_saved.append(idx)

            scores.append([
                idx,
                skmetrics.accuracy_score(v_true, v_predict),
                skmetrics.precision_score(v_true, v_predict, average='micro'),
                skmetrics.recall_score(v_true, v_predict, average='micro')
            ])

        # Provide labels after window length has passed
        if len(learning_window) == window_len:
            for x, y in learning_window:
                # Learn first sample at start of window
                if skipping == learn_skip:
                    started = True
                    knn.learn_one(x, y)
                    skipping = 0
                else:
                    skipping += 1
            learning_window = []

    if clusters:
        return pd.Series(v_predict, index=order_saved)
        
    return pd.DataFrame(scores, columns=['step', 'accuracy', 'precision', 'recall'])

Load features

In [ ]:
# TODO: refactor
def get_features_list(domains):
    features = []
    for dname, dataset in domains.items():
        names = pd.read_csv(dataset)
        names = names.columns.str.extract(r'([a-z]{2})_([a-z\_\-]+)')[1].unique()
        features.extend([f'{dname}_{col.strip("_")}' for col in names if not pd.isnull(col)])

    return features

def load_source(dataset: str, domain: str, row: dict):
    faults = {
        'A': {
            'normal': 'normal',
            'imbalance': 'imbalance',
            'horizontal-misalignment': 'misalignment',
            'vertical-misalignment': 'misalignment',
            'underhang-outer_race': 'outer race fault',
            'underhang-cage_fault': 'cage fault',
            'underhang-ball_fault': 'ball fault'
        },
        'B': {
            'normal': 'normal',
            'imbalance': 'imbalance',
            'horizontal-misalignment': 'misalignment',
            'vertical-misalignment': 'misalignment',
            'overhang-cage_fault': 'cage fault',
            'overhang-ball_fault': 'ball fault',
            'overhang-outer_race': 'outer race fault'
        }
    }
    placements = {
        'A': ['ax', 'ay', 'az'],
        'B': ['bx', 'by', 'bz']
    }

    RPM = 2500
    RPM_RANGE = 500
    features = pd.read_csv(dataset).fillna(0)

    # Choosing rpm range
    if row['rpm_limit']:
        features = features[features['rpm'].between(RPM - RPM_RANGE, RPM + RPM_RANGE, inclusive='both')]

    # Labeling anomaly severity levels
    target = re.search(r'([a-z]+)_?(\d+)?', row['target'])
    anomaly_severity = target.group(2) or '60'
    anomaly_severity = float(anomaly_severity) / 100

    # Choose measurement placement: A or B
    place = row['placement']
    axis = placements[place]
    features = features[features['fault'].isin(tuple(faults[place]))]
    features = models.fault_labeling(features, faults[place], anomaly_severity)

    columns = features.columns.str.startswith(tuple(axis))
    X = features[features.columns[columns]]

    # Select predicted variable column
    label = target.group(1)
    Y = features[label].astype('category')

    # Filter columns in feature domain with window size 2**14
    if domain == 'spectral':
        window_size = 2**14
        X = X.loc[:,X.columns.str.endswith(f'_{window_size}')]
        X.columns = X.columns.str.extract(r'(\w+)_\w+$')[0]

    # Calculate feature magnitudes from 3D vector
    feature_names = get_features_list({domain: dataset})
    result = pd.DataFrame()
    for name in feature_names:              
        # Remove prefix: temporal, spectral
        name = re.search(r'[a-z]+_([\w\_]+)', name).group(1)
        vector_dims = [f'{dim}_{name}' for dim in axis]
        result[name] = X[vector_dims].apply(np.linalg.norm, axis=1)
    X = result

    # Batch / Online hold-out (balance and event sequencing)
    train_size = 0.8

    # Shuffle order within severity level and order event with increasing severity
    groups = [
        df.sample(frac=1, random_state=10)
        for i, df in (
            features.sort_values(by='severity_level').groupby('severity_level')
        )
    ]
    rows = list(pd.concat(groups).index)
    
    X = X.loc[rows].reset_index(drop=True)
    Y = Y.loc[rows].reset_index(drop=True)

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, train_size=train_size, random_state=10
    )  
    X_train, X_test, Y_train, Y_test = (
        X_train.sort_index(), X_test.sort_index(),
        Y_train.sort_index(), Y_test.sort_index()
    )

    serevity_groups = pd.concat(groups)['severity_level'].reset_index(drop=True).sort_index()
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(np.arange(0, len(X_train)), serevity_groups.loc[X_train.index], color='red')
    ax.set_xlabel('Observations')
    ax.set_ylabel('Severity level')
    ax.grid()
    plt.show()

    return X_train, Y_train, X_test, Y_test   # WARNING: order matters

In [ ]:
dataset = {'temporal': TD_FEATURES, 'spectral': FD_FEATURES}
config = {'rpm_limit': False, 'placement': 'A', 'domain': DOMAIN, 'target': 'fault'}
XStream_fault, YStream_fault, _, YF1 = load_source(dataset[DOMAIN], config['domain'], config)
config = {'rpm_limit': False, 'placement': 'A', 'domain': DOMAIN, 'target': 'anomaly'}
XStream_anomaly, YStream_anomaly, _, YA1 = load_source(dataset[DOMAIN], config['domain'], config)

Evolution of faults and anomalies

Train

In [ ]:
a, b = discovery.plot_label_occurences(YStream_fault)

In [ ]:
a, b = discovery.plot_label_occurences(YStream_anomaly)

Test

In [ ]:
discovery.plot_label_occurences(YF1)

In [ ]:
discovery.plot_label_occurences(YA1)

Gradual learning
- 6 classes

In [ ]:
results = knn_online_learn(XStream_fault, YStream_fault, label='fault', window_len=1)
ax = results[['accuracy']].plot(
    grid=True, legend=False, figsize=(8, 4),
    xlabel='Sample', ylabel='Accuracy', title='Fault classes: 6, Window size: 1'
)
best = results.tail(1)
best

Gradual learning
- Binary classifier - anomaly

In [ ]:
results = knn_online_learn(XStream_anomaly, YStream_anomaly, label='anomaly_90', window_len=1)
ax = results[['accuracy']].plot(
    grid=True, legend=False, figsize=(8, 4),
    xlabel='Sample', ylabel='Accuracy', title='Fault classes: 1, Window size: 1'
)
best = results.tail(1)
best

Window learning
- Compare classification accuracies for window sizes in one graph: (1, 10, 50, 100, 250)
- Scenarios: fault, anomaly

In [ ]:
learning_window_lengths = (1, 10, 50, 100, 250)

fault_evolution = pd.DataFrame()
for n in tqdm(learning_window_lengths):
    results = knn_online_learn(XStream_fault, YStream_fault, label='fault', window_len=n)
    accuracy = results['accuracy']
    accuracy.index += n             # Starts learning after at least one window has been filled
    fault_evolution[str(n)] = accuracy

In [ ]:
ax = fault_evolution.fillna(0).plot(
    grid=True, legend=True, figsize=(8, 4),
    xlabel='Sample', ylabel='Accuracy', title='Faults: Label with delay'
)
fault_evolution.tail(1)

In [ ]:
anomaly_evolution = pd.DataFrame()
for n in tqdm(learning_window_lengths):
    results = knn_online_learn(XStream_anomaly, YStream_anomaly, label='anomaly_90', window_len=n)
    accuracy = results['accuracy']
    accuracy.index += n             # Starts learning after at least one window has been filled
    anomaly_evolution[str(n)] = accuracy

In [ ]:
ax = anomaly_evolution.fillna(0).plot(
    grid=True, legend=True, figsize=(8, 4),
    xlabel='Sample', ylabel='Accuracy', title='Anomaly: Label with delay'
)
anomaly_evolution.tail(1)

Missing labels - Faults

In [ ]:
window_len = 10
labels_skips = (0, 5, 15, 25, 50, 100)

fault_skip_evolution = pd.DataFrame()
for s in tqdm(labels_skips):
    results = knn_online_learn(XStream_fault, YStream_fault, label='fault', window_len=window_len, learn_skip=s)
    accuracy = results['accuracy']
    accuracy.index += len(XStream_fault) - len(accuracy)
    fault_skip_evolution[str(s)] = accuracy

In [ ]:
ax = fault_skip_evolution.fillna(0).plot(
    grid=True, legend=True, figsize=(8, 4),
    xlabel='Sample', ylabel='Accuracy', title=f'Faults (4 classes): Skip labels (out of {len(XStream_fault)} total), Window: {window_len}'
)
fault_skip_evolution.tail(1)

Missing labels - Anomaly

In [ ]:
anomaly_skip_evolution = pd.DataFrame()
for s in tqdm(labels_skips):
    results = knn_online_learn(XStream_anomaly, YStream_anomaly, label='anomaly_90', window_len=window_len, learn_skip=s)
    accuracy = results['accuracy']
    accuracy.index += len(XStream_anomaly) - len(accuracy)
    anomaly_skip_evolution[str(s)] = accuracy

In [ ]:
ax = anomaly_skip_evolution.fillna(0).plot(
    grid=True, legend=True, figsize=(8, 4),
    xlabel='Sample', ylabel='Accuracy', title=f'Anomaly: Skip labels (out of {len(XStream_anomaly)} total), Window: {window_len}'
)
anomaly_skip_evolution.tail(1)

Scatter plot - True labels vs. Predicted labels
- Faults
- Anomaly

In [ ]:
project_classifier_map_plot(
    filter_out_metadata_columns(XStream_fault.reset_index(drop=True)),
    YStream_fault.reset_index(drop=True),
    knn_online_learn(XStream_fault.reset_index(drop=True), YStream_fault.reset_index(drop=True), label='fault', window_len=1, learn_skip=0, clusters=True)
)
plt.show()

In [ ]:


def knn_model_setup(n):
    engine = neighbors.SWINN(
        dist_func=functools.partial(utils.math.minkowski_distance, p=2),
        seed=10
    )
    model = (
        preprocessing.MinMaxScaler() |
        neighbors.KNNClassifier(n_neighbors=n, engine=engine)
    )
    return model


def knn_accuracy_with_delays(X, y, delays):
    knn = knn_model_setup(5)

    evolution = defaultdict(dict)
    for delay in delays:
        steps = evaluate.iter_progressive_val_score(
            model=knn,
            dataset=stream.iter_pandas(X, y),
            metric=metrics.Accuracy(),
            step=100,
            delay=delay
        )
        for step in steps:
            step_num = step['Step']
            evolution[step_num]['Observation'] = step_num
            evolution[step_num][delay] = step['Accuracy'].get()

    evolution = (
        pd.DataFrame
        .from_records(list(evolution.values()))
        .set_index('Observation')
    )
    evolution.plot(
        grid=True, figsize=(15, 4), 
        marker='.', ylabel='Accuracy', 
        title='Accuracy with different delays'
    )
    return evolution


def knn_conf_matrix_plot(X, y):
    knn = knn_model_setup(5)
    #confmatrix = metrics.ConfusionMatrix()
    y_predictions = []

    for x, y_true in stream.iter_pandas(X, y):
        y_predict = knn.predict_one(x) or 0
        knn.learn_one(x, y_true)
        y_predictions.append(y_predict)
        # confmatrix.update(y_true, y_predict)

    cm = skmetrics.confusion_matrix(y, y_predictions)
    ax = sb.heatmap(cm, cbar=True, cmap='BuGn', annot=True, fmt='d')
    ax.set(xlabel='Prediction', ylabel='Truth')


def knn_visualize_classes(X, y):
    knn = knn_model_setup(5)

    y_predictions = []
    for xs, ys in stream.iter_pandas(X, y):
        y_predict = knn.predict_one(xs)
        knn.learn_one(xs, ys)
        y_predictions.append(y_predict)

    y_predictions = pd.Series(y_predictions)
    mismatch = models.project_classifier_map_plot(X, y, y_predictions)
    print(f'Error rate: {100 * (len(mismatch) / len(y)):.2f} %')

### kNN classifier (Faults)

In [ ]:
X, y = XStream_fault.loc[:,XStream_fault.columns != 'fault'], YStream_fault

In [ ]:
evolution = knn_accuracy_with_delays(X, y, (1, 50, 100, 250))
plt.show()
evolution.tail(1)

In [ ]:
evolution[evolution[2] > 0.5] 

### Classification report (Faults)

In [ ]:
evaluate.progressive_val_score(
    model=knn_model_setup(5),
    dataset=stream.iter_pandas(X, y),
    metric=metrics.ClassificationReport()
)

### Confusion matrix (Faults)

In [ ]:
knn_conf_matrix_plot(X, y)

### Visualize clusters of nearest neighbors (Faults)

In [ ]:
x_scaled = X.reset_index(drop=True)
y_scaled = y.reset_index(drop=True)
x_scaled[x_scaled.columns] = MinMaxScaler().fit_transform(x_scaled)
knn_visualize_classes(x_scaled, y_scaled)

### kNN classifier (Anomaly)

In [ ]:
X, y = XStream_anomaly.loc[:,XStream_fault.columns != 'fault'], YStream_anomaly

In [ ]:
knn_accuracy_with_delays(X, y, (1, 50, 100, 250))
plt.show()

### Classification report (Anomaly)

In [ ]:
evaluate.progressive_val_score(
    model=knn_model_setup(5),
    dataset=stream.iter_pandas(X, y),
    metric=metrics.ClassificationReport()
)

### Confusion matrix (Anomaly)

In [ ]:
knn_conf_matrix_plot(X, y)

### Visualize clusters of nearest neighbors (Anomaly)

In [ ]:
x_scaled = X.reset_index(drop=True)
y_scaled = y.reset_index(drop=True)
x_scaled[x_scaled.columns] = MinMaxScaler().fit_transform(x_scaled)
knn_visualize_classes(x_scaled, y_scaled)